### Importations des packages

In [2]:
import requests
import pandas as pd
import numpy as np 
import s3fs
from scipy.stats import zscore
import matplotlib.pyplot as plt
from tqdm import tqdm
from helpers import *
pd.set_option('display.float_format', '{:.2f}'.format)


Plan

0. Ce qu'on veut faire sur ce ntb
1. Data extraction
2. Data preparation et calculs


# 0. Ce qu'on veut faire sur ce ntb

Notre but, comme évoqué sur readme, est de essayer d'expliquer la charge de la gare en nombre de voyageurs en 2022. Pour cela, on veut avoir pour chaque gare les données:

type 1: géographiques, les fréquentations des gares issues du site https://ressources.data.sncf.com/pages/accueil/

type 2: les statistiques des données communales issues du site https://www.unehistoireduconflitpolitique.fr/telecharger.html

On veut avant tout avoir le maximum des données qui pourraient potentiellement expliquer la charge de la gare et après se baser sur les critères de selection de variables dans notre modèle d'interpretation, regarder quelles sont les variables pertinentes etc. Donc ici prenons le temps d'obtenir les variables intéressantes, on va vous guider dans notre reflexion.

# 1. Data extraction

Dans cette section on veut récupérer les données pertinentes avec API et les données du site, et intéragir avec S3

Regardons d'abord comment extraire les données de type 1 avec une API de SNCF:


In [3]:
station_geo_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="gares-de-voyageurs",
    select="""nom as nom_gare,
                           position_geographique,
                           codeinsee,
                           codes_uic as uic""",
)

station_freq_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="frequentation-gares",
    select="""total_voyageurs_2022,
                           
                           nom_gare,
                           code_uic_complet as uic """,
)

nb of stations downloaded: 2881, from table gares-de-voyageurs
nb of stations downloaded: 3010, from table frequentation-gares


In [4]:
#fusion de ces 2 tables
station_geo_data = station_geo_data.rename(
    columns={
        "position_geographique.lon": "lon_gare",
        "position_geographique.lat": "lat_gare",
    }
)
station_geo_data = station_geo_data.drop("position_geographique", axis=1)

merged = station_freq_data.merge(
    station_geo_data.drop("nom_gare", axis=1), on=["uic"], how="left"
)
merged = merged[merged["total_voyageurs_2022"] > 0]#avoir les gares ouvertes


### Maintenant, regardons comment importer des données de type 2 (communales) avec les statistiques qui nous intéressent dans notre analyse

Pour cela, on a besoin de se connecter au service de stockage S3, car ces fichiers sont disponibles sous format zip/csv sur le site indiqué en haut, cad on les a stocké manuellement sur S3 en dézippant

In [5]:
s3=s3_connection()

connection successful


On importe les données communales. Nous avons des fichiers de données sur le taux de personnes diplômées par commune, la population et le revenu par habitant.

1. Revenus par habitant et population par commune

In [6]:
columns_to_select = ["codecommune", "nomcommune", "pop2022", "revmoy2022"]
dtype_spec = {'codecommune': 'str'}

path_revcommunes = "clichere/diffusion/revcommunes.csv"
revcommunes = s3.read_csv_from_s3(path_revcommunes, columns_to_select, dtype_spec)

revcommunes.sample(2)


,codecommune,nomcommune,pop2022,revmoy2022
16045,40253,SAINT-CRICQ-CHALOSSE,591,14863.28
20594,53175,PARNE-SUR-ROC,1380,17573.42


2. Taux de diplome par commune

In [7]:
columns_to_select = ["codecommune", "pbac2022", "psup2022"]
dtype_spec = {'codecommune': 'str'}

path_dipcommunes = "clichere/diffusion/diplomescommunes.csv"
dipcommunes = s3.read_csv_from_s3(path_dipcommunes, columns_to_select, dtype_spec)

dipcommunes.sample(2)


,codecommune,pbac2022,psup2022
34776,81059,0.70,0.29
24697,60057,0.39,0.25


3. Catégories socio-professionnelles

In [8]:
columns_to_select = ["codecommune","pagri2022", "pempl2022", "pcadr2022", "pouvr2022", "pchom2022", "pindp2022","ppint2022"]
dtype_spec = {'codecommune': 'str'}

path_cspcommunes = "clichere/diffusion/cspcommunes.csv"
cspcommunes = s3.read_csv_from_s3(path_cspcommunes, columns_to_select, dtype_spec)

cspcommunes.sample(2)

,codecommune,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
3020,08483,0.00,0.00,0.06,0.42,0.33,0.18,0.00
36655,88301,0.09,0.00,0.17,0.53,0.00,0.21,0.00


4. Fusion et export des fichiers de données

In [9]:
intermediaire_comm = pd.merge(revcommunes, dipcommunes, on='codecommune', how='inner')
#final_comm est une table final pour les donées de type 2
final_comm = pd.merge(intermediaire_comm, cspcommunes, on="codecommune", how="inner")

final_comm.sample(2)

,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
14924,38245,MONTAGNE,251,18672.26,0.50,0.29,0.00,0.00,0.04,0.36,0.34,0.26,0.09
30047,72195,MEZERAY,1992,13827.60,0.37,0.19,0.03,0.01,0.18,0.27,0.17,0.33,0.04


la table finale de type 2 est prête, donc il faut qu'elle puisse être utilisée par une autre personne qui ne va pas refaire ces étapes, donc il faut stocker sur s3
## ATTENTION, ne pas executer ce code, car il write sur le dossier d une autre personne ( pas de permissions)

In [8]:
output_path1 = 'clichere/diffusion/final_comm.parquet'
output_path2 = 'clichere/diffusion/revcommunes.parquet'
output_path3 = 'clichere/diffusion/dipcommunes.parquet'
output_path4 = 'clichere/diffusion/cspcommunes.parquet'

s3.from_pandas_to_parquet_store_in_s3(final_comm, output_path1)
s3.from_pandas_to_parquet_store_in_s3(revcommunes, output_path2)
s3.from_pandas_to_parquet_store_in_s3(dipcommunes, output_path3)
s3.from_pandas_to_parquet_store_in_s3(cspcommunes, output_path4)


## 2. Data préparation et calculs

Ici on va fusionner les 2 types de données et travailler sur les fusions, cleaning des données et calculs supplémentaires, tout pour avoir notre table clean et propre et prête pour notre analyse

In [59]:
regional_stat=s3.get_tables_from_s3("clichere/diffusion/final_comm.parquet")
regional_stat.sample(2)

,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
36105,90082,AUTRECHENE,301,20278.04,0.53,0.41,0.06,0.07,0.33,0.00,0.55,0.00,0.01
34924,87072,GLANGES,495,14833.10,0.68,0.40,0.24,0.00,0.16,0.20,0.04,0.37,0.00


On supprime les communes qui n'ont pas d'habitants en 2022 ce qui compte pour 5% de la table.

Moins de 50 gares sont supprimés en utilisant le filtre sur le revenu moyen et prct du bac.

Ces informations peuvent manquer car certaines communes ont fusionné avant 2022 et donc la donnée n'est plus disponible pour ces communes.

In [60]:
old_nb_row=regional_stat.shape[0]
regional_stat = regional_stat[
    (regional_stat["pop2022"] > 0)
    & (regional_stat["revmoy2022"] > 0)
    & (regional_stat["pbac2022"].notna())
]
new_nb_row=regional_stat.shape[0]
print("prct communes supprimées",np.round((old_nb_row-new_nb_row)/old_nb_row,3)  )

prct communes supprimées 0.052


Ensuite, on fusionne les 2 types de données ensemble en se basant sur le code insee qui est la vraie clé de reconciliation 
On s'assure que toutes les données sont assurées et qu'il y a plus de NA avec la commande *final.isna().sum()*

In [68]:
fusion=merged.merge(regional_stat,left_on="codeinsee",right_on="codecommune",how="left")
#filtrer sur les communes non identifiés (car bases des données différentes), on a pas le moyen de les réidentifier 
final=fusion[fusion["pop2022"].notna()]
print('prct gares supprimés', np.round((fusion.shape[0]-final.shape[0])/fusion.shape[0],2) )
final.sample(2)

prct gares supprimés 0.04


,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022
2109,1,Margival,87271882,02464,3.40,49.44,02464,MARGIVAL,393.00,14886.32,0.29,0.29,0.00,0.26,0.00,0.30,0.34,0.11,0.01
2508,48263,Eymoutiers Vassivière,87592121,87064,1.74,45.74,87064,EYMOUTIERS,2124.00,13791.98,0.55,0.29,0.00,0.14,0.09,0.35,0.35,0.07,0.00


## Question soulevée

Une question se pose concernant l'analyse des données géographiques et communales disponibles.

### Données disponibles :

- Nous avons toutes les données **communales**, ce qui est très utile.
- Cependant, pour les **données géographiques**, nous ne disposons que des informations suivantes :
  - **Latitude** et **Longitude** des communes
  - **Nom de la commune**

### Problème :

Les informations géographiques (latitude, longitude) sont parfois insuffisantes pour une analyse plus poussée, car elles ne sont pas forcément très parlantes sans un contexte supplémentaire.

---

### Ce qui est important à considérer :

- **La population** des communes est bien sûr un facteur crucial.
- **La densité des gares** dans la région joue également un rôle significatif. En effet, une gare éloignée des autres gares pourrait potentiellement attirer un grand nombre de voyageurs de la région, ce qui peut influencer le transport et les flux de personnes.

### Proposition de solution :

- Une approche possible pour enrichir les données géographiques serait de calculer la **distance minimale** de chaque gare à son **voisin le plus proche**. 
- Cela permettrait d'obtenir une idée plus précise de l'accessibilité de chaque gare, ce qui pourrait être très utile pour certaines analyses.

Bien entendu, l'impact de la gare éloignée dépendra de nombreux facteurs, tels que la **taille de la ville**, la **commune**, ou même la **région** dans son ensemble. Mais, pour commencer, calculons déjà la distance minimale entre chaque gare et son voisin le plus proche.





In [69]:
#pour cela il faut faire cross join et on va obtenir pour chaque obsérvation toutes les obsérvations de la table ( table de nb de lignes  N*N)
cross_table_for_distance_calculation = final[["uic", "lon_gare", "lat_gare"]].merge(
    final[["uic", "lon_gare", "lat_gare"]], how="cross"
)
#nommer les variables 
(
    lat1,
    lon1,
    lat2,
    lon2,
) = (
    cross_table_for_distance_calculation["lat_gare_x"],
    cross_table_for_distance_calculation["lon_gare_x"],
    cross_table_for_distance_calculation["lat_gare_y"],
    cross_table_for_distance_calculation["lon_gare_y"],
)
#appliquer la formule haversine
cross_table_for_distance_calculation["dist_closest_station_km"] = haversine_vectorized(
    lat1, lon1, lat2, lon2
)
#enlever les gares par rapport à elles-mêmes ( N obsérvations)
cross_table_for_distance_calculation = cross_table_for_distance_calculation[
    cross_table_for_distance_calculation["uic_x"]
    != cross_table_for_distance_calculation["uic_y"]
]
#grouper par les gares et sortir les indices de distance minimale
idx = cross_table_for_distance_calculation.groupby(["uic_x"])[
    "dist_closest_station_km"
].idxmin()
#sortir toutes les infos sur ces gares
result = cross_table_for_distance_calculation.loc[idx].reset_index(drop=True)
#merger pour avoir l'ensemble de l'info
final = final.merge(
    result[["uic_x", "dist_closest_station_km"]].rename(columns={"uic_x": "uic"}),
    on="uic",
    how="inner",
)
final.sample(2)

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022,dist_closest_station_km
2676,137990,Roubaix,87286732,59512,3.16,50.70,59512,ROUBAIX,99044.00,8906.67,0.38,0.24,0.00,0.06,0.11,0.20,0.33,0.29,0.14,1.25
1628,309631,Saint-Vallier sur Rhône,87761130,26333,4.81,45.19,26333,ST-VALLIER,3782.00,13879.23,0.50,0.29,0.01,0.10,0.07,0.22,0.21,0.38,0.11,12.39


pour la densité des gares par région il serait juste interessant de regarder combien de gares il y a dans la même commune, donc sortons cette info 

In [ ]:

communes_count=final.groupby('nomcommune').agg({'nomcommune': ['count']})
communes_count["commune"]=communes_count.index
communes_count=communes_count.reset_index(drop=True)
communes_count.columns = ["nb_stations_same_commune","nomcommune"]
final=final.merge(communes_count,on="nomcommune",how="inner")


La **commune** peut être considérée comme une petite unité, similaire à une **observation** dans notre analyse. En effet, la majorité des gares se trouvent dans une seule commune, comme l'indique la variable **nb_stations_same_communes**.

### Objectif :

Pour réaliser une **visualisation** ou un **modèle**, il est nécessaire de regrouper les communes en **régions**. Nous allons nous appuyer sur les données géographiques disponibles pour effectuer ce regroupement.

### Informations disponibles pour chaque commune :

1. **Latitude** et **Longitude** (données géographiques)
2. **Nom de la commune** (comme dernier recours, si la latitude et longitude sont introuvables)

---

### Méthodologie proposée :

1. **Utiliser les données géographiques** (latitude et longitude) pour identifier la région associée à chaque commune.
   
   - Nous allons envoyer ces informations à l'**API gouvernementale** (Gouv.fr) qui peut retourner la région correspondante.

2. **Cas où les coordonnées géographiques sont manquantes** :
   
   - Si les informations géographiques (latitude, longitude) sont introuvables ou incorrectes, nous utiliserons le **nom de la commune** comme dernier recours pour déterminer la région.

---

### Variables à générer :

- **Région** : Une variable catégorielle qui devrait discriminer le plus possible la **charge dans les gares**. Cette région sera utilisée pour analyser la densité de gares et l'accessibilité dans chaque zone.
   


In [74]:
#appelons cet api, va tourner un peu 
french_regions=gouv_api_addresses(final)

In [76]:
final["regions"]=french_regions
#voilà les résultats de l'api,tout est bien renseigné
print(final["regions"].unique())
final.sample(2)


['Pays de la Loire' 'Île-de-France' 'Hauts-de-France'
 "Provence-Alpes-Côte d'Azur" 'Occitanie' 'Auvergne-Rhône-Alpes'
 'Bourgogne-Franche-Comté' 'Centre-Val de Loire' 'Normandie'
 'Nouvelle-Aquitaine' 'Grand Est' 'Bretagne']


,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,...,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022,dist_closest_station_km,regions
829,18333290,Le Bourget,87271395,93029,2.43,48.93,93029,DRANCY,74630.00,13011.62,...,0.23,0.00,0.08,0.09,0.21,0.36,0.26,0.10,1.77,Île-de-France
2011,68061,Malansac,87476689,56123,-2.30,47.68,56123,MALANSAC,2278.00,16117.42,...,0.25,0.00,0.04,0.08,0.25,0.33,0.29,0.12,11.52,Bretagne


Tout est bon! On a les données géographiques et économiques, exportons sur S3 et vérifions la connection
#### ATTENTION, on invite le lecteur à run la cellule après avec la fonction **get_tables_from_s3**

In [78]:
s3.from_pandas_to_parquet_store_in_s3(final,"aayrapetyan/diffusion/final_table.parquet")

In [79]:
final=s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")
final

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,...,psup2022,pagri2022,pindp2022,pcadr2022,ppint2022,pempl2022,pouvr2022,pchom2022,dist_closest_station_km,regions
0,40825,Abbaretz,87481614,44001,-1.52,47.55,44001,ABBARETZ,2257.00,12567.25,...,0.17,0.06,0.02,0.05,0.14,0.31,0.42,0.04,9.75,Pays de la Loire
1,177092,Achères Grand Cormier,87386052,78551,2.09,48.96,78551,SAINT-GERMAIN-EN-LAYE,39172.00,41601.99,...,0.64,0.00,0.04,0.39,0.27,0.23,0.07,0.06,1.97,Île-de-France
2,80648,Achiet-le-Grand,87342048,62005,2.78,50.13,62005,ACHIET-LE-GRAND,888.00,15813.03,...,0.34,0.00,0.00,0.11,0.35,0.16,0.38,0.08,4.22,Hauts-de-France
3,32800,Agay,87757559,83118,6.86,43.43,83118,SAINT-RAPHAEL,37114.00,26637.97,...,0.34,0.00,0.13,0.13,0.25,0.35,0.13,0.10,1.80,Provence-Alpes-Côte d'Azur
4,11325,Aigues-Mortes,87775858,30003,4.19,43.57,30003,AIGUES-MORTES,8076.00,20798.90,...,0.20,0.01,0.14,0.08,0.21,0.39,0.17,0.11,5.56,Occitanie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2803,176599,Weyersheim,87213678,67529,7.80,48.72,67529,WEYERSHEIM,3365.00,21737.75,...,0.26,0.01,0.15,0.23,0.33,0.22,0.07,0.03,2.10,Grand Est
2804,30849,Willer-sur-Thur,87182584,68372,7.07,47.84,68372,WILLER-SUR-THUR,1735.00,18620.66,...,0.13,0.00,0.09,0.09,0.23,0.24,0.34,0.04,1.82,Grand Est
2805,37020,Wimille - Wimereux,87317123,62894,1.61,50.76,62894,WIMILLE,4062.00,19524.12,...,0.28,0.01,0.07,0.15,0.27,0.21,0.30,0.06,4.08,Hauts-de-France
2806,38618,Ygos-Saint-Saturnin,87671487,40333,-0.74,43.98,40333,YGOS-SAINT-SATURNIN,1490.00,15004.68,...,0.28,0.00,0.08,0.11,0.17,0.33,0.32,0.09,6.68,Nouvelle-Aquitaine
